In [1]:
%matplotlib inline
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
datadir = '../../outputdata/regridded/'
ds = xr.open_dataset(datadir+'C384_wind_16days.nc', decode_times=False)

In [3]:
ds

<xarray.Dataset>
Dimensions:  (time: 16, x: 360, y: 180)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 -175.5 -174.5 -173.5 ...
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 -89.5 -89.5 -89.5 -89.5 ...
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
Dimensions without coordinates: x, y
Data variables:
    u700     (time, y, x) float64 0.01942 0.01936 0.01929 0.01921 0.01912 ...
    v700     (time, y, x) float64 -0.003344 -0.003345 -0.003345 -0.003346 ...

In [4]:
u = ds['u700'].values
v = ds['v700'].values
lat = ds['lat'].isel(x=0).values

In [5]:
def Liapunov(u, v, lat, dlat=1, dlon=1):
    s_lat = 110.574e3 * dlat
    s_lon = 111.320e3 * np.cos(lat/180*np.pi) * dlon
    du = (np.roll(u,1,axis=-1)-np.roll(u,-1,axis=-1))
    dv = (np.roll(v,1,axis=-2)-np.roll(v,1,axis=-2))
    L = (np.abs(du)+np.abs(dv))/(s_lon+s_lat)[:,np.newaxis]
    return L

In [6]:
L = Liapunov(u, v, lat)*1e6
L.shape

(16, 180, 360)

In [11]:
vmin,vmax = [0,40]

for t in range(16):
    print(t,end=' ')
    fig = plt.figure(figsize=[8,6])
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()

    im = ax.pcolormesh(ds['lon'].values,ds['lat'].values,L[t,:,:],
                  cmap='jet',vmin=vmin, vmax=vmax)
    ax.set_title('day = {0}'.format(t-7), fontsize=12)

    plt.colorbar(im, orientation='vertical', shrink=0.5,
                 ticks=np.linspace(vmin,vmax,5)).\
                 set_label("Lyapunov exponent (1e-6/s)",fontsize=10)

    fig.savefig('Lyapunov_iday{0:02}'.format(t+1),dpi=200)
    plt.close() # don't show in this notebook

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 